<a href="https://colab.research.google.com/github/wingated/cs473/blob/main/labs/cs473_lab_week_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a><p><b>After clicking the "Open in Colab" link, copy the notebook to your own Google Drive before getting started, or it will not save your work</b></p>

# BYU CS 473 Lab Week 2

## Introduction:
Welcome to your first lab for CS 473, Advanced Machine Learning.

In machine learning, models often predict *unnormalized log probabilities*. These must often be converted into regular probabilities.

In this lab, you will explore the log-sum-exp function, which is described in the text (Sec. 2.5.4).  You will code up several variants of the function, and compare their performance.

# Part 1: Logsumexp
---
## Setup: The Iris Dataset
We'll begin by downloading the Iris dataset. The iris dataset is a simple, but very famous, dataset introduced to the world by RA Fisher (the “father” of modern statistics”) in 1939. The dataset has five columns:
* sepal length (cm)
* sepal width (cm)
* petal length (cm)
* petal width (cm)
* class

In order to get logits to play with, we'll first train a multinomial logistic regression model (Sec. 2.5.3).  This model naturally outputs logits.

In [19]:
import datasets
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

ds = datasets.load_dataset( "scikit-learn/iris" )

df = pd.DataFrame( ds['train'] )

X = np.array( df[['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']] )
Y = np.array( LabelEncoder().fit_transform( df['Species'] ) )

In [20]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression().fit(X,Y)

W = model.coef_
b = model.intercept_

b = np.reshape( b, (3,1))

logits = np.dot( W, X.T ) + b

---
## Exercise 1: convert logits to probabilities

Since our model outputs logits, they must be converted. To do this, we'll use the softmax function.

In [21]:
def softmax( logits ):
    # logits is a numpy matrix of d x N
    # where
    #   d is the number of classes
    #   N is the number of data points
    # use equation 2.99 (see also Eq. 2.94)

    exp_logits = np.exp(logits)
    Z_a = np.sum(exp_logits, axis=0, keepdims=True)
    probabilities = exp_logits / Z_a

    return probabilities

In [23]:
# Test case 1: probs[:,0]
probs = softmax(logits)
print("probs[:,0] =", probs[:,0])
print("Expected:   [9.81803910e-01, 1.81960759e-02, 1.43430317e-08]")
print()

# Test case 2: probs[:,120]
print("probs[:,120] =", probs[:,120])
print("Expected:     [5.49519371e-06, 2.38812718e-02, 9.76113233e-01]")
print()

# Verify probabilities sum to 1
print("Sum of probs[:,0] =", np.sum(probs[:,0]))
print("Sum of probs[:,120] =", np.sum(probs[:,120]))
print("(Should both be 1.0)")


probs[:,0] = [9.81747643e-01 1.82523424e-02 1.43429404e-08]
Expected:   [9.81803910e-01, 1.81960759e-02, 1.43430317e-08]

probs[:,120] = [5.54234723e-06 2.39030160e-02 9.76091442e-01]
Expected:     [5.49519371e-06, 2.38812718e-02, 9.76113233e-01]

Sum of probs[:,0] = 0.9999999999999999
Sum of probs[:,120] = 1.0
(Should both be 1.0)


### test cases
probs = softmax( logits )
probs[:,0]
#### array([9.81803910e-01, 1.81960759e-02, 1.43430317e-08])
probs[:,120]
#### array([5.49519371e-06, 2.38812718e-02, 9.76113233e-01])

---
## Exercise 2: convert logits to probabilities

Now, code up the logsumexp function.  What test cases should you use for this function?

In [24]:
def logsumexp( logits ):
    # logits is a numpy matrix of d x N
    # where
    #   d is the number of classes
    #   N is the number of data points
    # use equation 2.100

    m = np.max(logits, axis=0, keepdims=True)
    shifted_logits = logits - m
    log_sum_exp_shifted = np.log(np.sum(np.exp(shifted_logits), axis=0, keepdims=True))

    return m + log_sum_exp_shifted

In [25]:
# test cases
probs = logsumexp( logits )
probs[:,0]
print("probs[:,0] = ", probs[:,0])
print()


probs[:,0] =  [7.36063027]



What should be printed??

[7.36063027]

---
## Exercise 3: explore underflow / overlow

First, code up a function that compares two distributions. This can be anything you want; you may consider things like the MSE.

In [26]:
def compare_probs( probs1, probs2 ):
  if probs2.shape[0] == 1:  # logsumexp returns (1, N)
      probs2_as_probs = np.exp(logits - probs2)
      return np.mean((probs1 - probs2_as_probs) ** 2)
  else:
      return np.mean((probs1 - probs2) ** 2)

In [27]:
probs1 = softmax( logits )
probs2 = logsumexp( logits )
compare_probs( probs1, probs2 )

np.float64(1.834799520780957e-32)

Now, see what happens if you add (or subtract) a constant from logits. How big must the constant be before things start going haywire?

In [28]:
print("Testing overflow/underflow with different constants: ")
constants = [0, 100, 500, 700, 710, 720]

for C in constants:
  print(f"\nTesting C = {C}:")

  try:
      probs1 = softmax(logits + C)
      has_overflow_softmax = np.any(np.isinf(probs1)) or np.any(np.isnan(probs1))
      print(f"  Softmax: overflow/nan = {has_overflow_softmax}")
      if has_overflow_softmax:
          print(f"    Sample values: {probs1[:,0]}")
  except Exception as e:
      print(f"  Softmax: Failed with exception: {e}")

  try:
      probs2 = logsumexp(logits + C)
      has_overflow_logsumexp = np.any(np.isinf(probs2)) or np.any(np.isnan(probs2))
      print(f"  Logsumexp: overflow/nan = {has_overflow_logsumexp}")
      if not has_overflow_logsumexp:
          print(f"    Sample values: {probs2[:,0]}")
  except Exception as e:
      print(f"  Logsumexp: Failed with exception: {e}")
# etc.

Testing overflow/underflow with different constants: 

Testing C = 0:
  Softmax: overflow/nan = False
  Logsumexp: overflow/nan = False
    Sample values: [7.36063027]

Testing C = 100:
  Softmax: overflow/nan = False
  Logsumexp: overflow/nan = False
    Sample values: [107.36063027]

Testing C = 500:
  Softmax: overflow/nan = False
  Logsumexp: overflow/nan = False
    Sample values: [507.36063027]

Testing C = 700:
  Softmax: overflow/nan = False
  Logsumexp: overflow/nan = False
    Sample values: [707.36063027]

Testing C = 710:
  Softmax: overflow/nan = True
    Sample values: [nan nan  0.]
  Logsumexp: overflow/nan = False
    Sample values: [717.36063027]

Testing C = 720:
  Softmax: overflow/nan = True
    Sample values: [nan nan  0.]
  Logsumexp: overflow/nan = False
    Sample values: [727.36063027]


/tmp/ipython-input-2289847087.py:8: RuntimeWarning: overflow encountered in exp
  exp_logits = np.exp(logits)
/tmp/ipython-input-2289847087.py:10: RuntimeWarning: invalid value encountered in divide
  probabilities = exp_logits / Z_a


Now convert the logits to 16-bit precision, and re-run your experiments. Analyze the differences you see (2-3 sentences).

In [29]:
logits = logits.astype( np.float16 )

constants_16 = [0, 10, 15, 20, 25, 30]


for C in constants_16:
    print(f"\nTesting C = {C} with float16:")

    try:
        probs1 = softmax(logits + C)
        has_overflow_1 = np.any(np.isinf(probs1)) or np.any(np.isnan(probs1))
        print(f"  Softmax: overflow/nan = {has_overflow_1}")
        if has_overflow_1:
            print(f"    First few values: {probs1[:,0]}")
    except Exception as e:
        print(f"  Softmax: Exception - {e}")

    try:
        probs2 = logsumexp(logits + C)
        has_overflow_2 = np.any(np.isinf(probs2)) or np.any(np.isnan(probs2))
        print(f"  Logsumexp: overflow/nan = {has_overflow_2}")
        if has_overflow_2:
            print(f"    First few values: {probs2[:,0]}")
    except Exception as e:
        print(f"  Logsumexp: Exception - {e}")

# Analysis
print("\nAnalysis:")


Testing C = 0 with float16:
  Softmax: overflow/nan = False
  Logsumexp: overflow/nan = False

Testing C = 10 with float16:
  Softmax: overflow/nan = True
    First few values: [nan nan  0.]
  Logsumexp: overflow/nan = False

Testing C = 15 with float16:
  Softmax: overflow/nan = True
    First few values: [nan nan  0.]
  Logsumexp: overflow/nan = False

Testing C = 20 with float16:
  Softmax: overflow/nan = True
    First few values: [nan nan  0.]
  Logsumexp: overflow/nan = False

Testing C = 25 with float16:
  Softmax: overflow/nan = True
    First few values: [nan nan nan]
  Logsumexp: overflow/nan = False

Testing C = 30 with float16:
  Softmax: overflow/nan = True
    First few values: [nan nan nan]
  Logsumexp: overflow/nan = False

Analysis:
(Your analysis here)


/tmp/ipython-input-2289847087.py:8: RuntimeWarning: overflow encountered in exp
  exp_logits = np.exp(logits)
/tmp/ipython-input-2289847087.py:10: RuntimeWarning: invalid value encountered in divide
  probabilities = exp_logits / Z_a


### Analysis

With float64 precision, softmax remained stable up to C=700 and failed at C=710. With float16 precision, softmax failed much earlier at C=10, showing how reduced precision dramatically decreases numerical stability. However, logsumexp remained stable in both cases, demonstrating the effectiveness of the numerical stability technique in equation 2.100. The dramatic difference between failure points shows that float16's limited range makes overflow much more likely even with modest increases to the input values.

---
## Exercise 4: cleanly compute log probabilities

Sometimes, we want to compute log probabilities (which are different from logits), but we want to do so "cleanly", ie, while avoiding overflow / underflow. First, mathematically figure out what the log of the softmax is (ie, take the log of eq. 2.99), and then combine it with insights from coding up the logsumexp function. Hint: at the end of the day, you will simply shift each column by a per-column constant!

In [ ]:
def log_logsumexp( logits ):
    # logits is a numpy matrix of d x N
    # where
    #   d is the number of classes
    #   N is the number of data points

    # your code here
    pass

---
# Part 2: Probability Fundamentals

For the following exercises, you are encouraged to work both by hand and by code however makes the most sense.



## Exercise 1a: Joint Probability Distributions

You are given the following two binary variables, X and Y, that can each take on the values 0 or 1. Assuming X and Y are independent, calculate the joint probability table (2x2 table for P(X, Y)). Display as a numpy array.

P(X=0) = 0.6

P(X=1) = 0.4

P(Y=0) = 0.7

P(Y=1) = 0.3


In [ ]:
# Your code here


Next, compute the following conditional probabilities:

P(X=0|Y=0) = ?

P(X=0|Y=1) = ?

P(Y=0|X=0) = ?

P(Y=0|X=1) = ?

In [ ]:
# Your code here


Compare the result of these conditional probabilities to the original marginal probabilities given. What does this say about the relationship between variable dependence and using conditional probabilities? Write 1-2 sentences.

(Your answer here)

## Exercise 1b: Joint Probability Distributions

Now consider this joint distribution:

|  | $Y = 0$ | $Y = 1$|
| :------- | :------: | -------: |
| $X = 0$  | 0.45  | 0.10  |
| $X = 1$  | 0.25  | 0.20  |

First, compute the marginals from the joint table

In [ ]:
joint_prob_table = np.array([[0.45, 0.10],
                             [0.25, 0.20]])

# P(X=0) = ?
# P(X=1) = ?
# P(Y=0) = ?
# P(Y=1) = ?

# Your answer here


Compute the same conditional probabilities as above:

P(X=0|Y=0) = ?

P(X=0|Y=1) = ?

P(Y=0|X=0) = ?

P(Y=0|X=1) = ?

In [ ]:
# P(X=0|Y=0) = ?
# P(X=0|Y=1) = ?
# P(Y=0|X=0) = ?
# P(Y=0|X=1) = ?

# Your answer here


Check if the independence property $P(X, Y) = P(X)P(Y)$ holds for any cell.

In [ ]:
# Your answer here


Compare P(X=0|Y=0) to P(X=0|Y=1), and discuss what this says about the dependence between these variables (1-2 sentences).

(Your answer here)

<br>

---
## Exercise 2: Bayes Theorem

After your yearly checkup, the doctor has bad news and good news. The bad news is that you tested positive
for a serious disease, and that the test is 99% accurate (i.e., the probability of testing positive given that you
have the disease is 0.99, as is the probability of testing negative given that you don’t have the disease). The
good news is that this is a rare disease, striking only one in 10,000 people. What are the chances that you
actually have the disease? (Show your calculations as well as giving the final result.)

*Hint: write out the variables you know, and think about what you'll need to calculate to find the final answer

In [ ]:
# Your answer/work here
